In [45]:
import utilities
import pandas as pd
import os

If it's the first time we run the code, we'll download the data from bigquery and create a local copy

In [46]:
dataFileName = "data.json"
if (not os.path.exists(dataFileName)):
    data = utilities.retrieveBigQueryData("bigquery-public-data:google_analytics_sample.ga_sessions_20170801")
    data.to_json(dataFileName, orient="records")

Let's import the data

In [47]:
data = pd.read_json(dataFileName)

Let's check the size of the data

In [189]:
print("Rows: ", data.shape[0])
print("Columns: ", data.shape[1])

Rows:  2556
Columns:  59
